In [8]:
# add this package to path
sys.path.insert(0, r'C:\Users\pasca\Dropbox\DataScienceAndProgramming\Projects\tox-block\tox-block')
from tox_block.config import config
from tox_block.data_processing import preprocessors as pp
from tox_block.data_processing import data_handling as dh
from tox_block.model import lstm_multi_label
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import re
import joblib
from tensorflow.keras.models import load_model

In [2]:
X_train, y_train = dh.load_data_multi_labels(config.TRAINING_DATA_FILE)
X_test, y_test = dh.load_data_multi_labels(config.TESTING_DATA_FILE)

In [4]:
ts = pp.TokenSequencer()
X_train_seq = ts.fit_transform(X_train)
X_test_seq = ts.transform(X_test)

In [5]:
em = dh.get_embedding_matrix(ts)

In [6]:
model = lstm_multi_label.get_model(embedding_matrix=em)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 100)          40400     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)    

In [7]:
history = model.fit(X_train_seq, y_train.to_numpy(), 
                    batch_size=config.BATCH_SIZE, 
                    epochs=config.EPOCHS, 
                    validation_split=config.VALIDATION_SPLIT,
                    callbacks=lstm_binary_label.callbacks_list, 
                    #class_weight=config.CLASS_WEIGHT,
                    )

Train on 143613 samples, validate on 15958 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
143584/143613 [============================>.] - ETA: 0s - loss: 0.0586 - acc: 0.9796
Epoch 00001: val_loss improved from inf to 0.05048, saving model to C:\Users\pasca\Dropbox\DataScienceAndProgramming\Projects\tox-block\tox-block\tox_block\trained_models/checkpoint_BL.h5
143613/143613 [==============================] - 1009s 7ms/sample - loss: 0.0586 - acc: 0.9796 - val_loss: 0.0505 - val_acc: 0.9809


In [8]:
model.evaluate(X_test_seq,y_test)

153164/153164 [==============================] - 203s 1ms/sample - loss: 0.2230 - acc: 0.9168


[0.2229632776401705, 0.91677]

In [9]:
y_pred = model.predict(X_test_seq)

In [10]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
print(f"ACC: {accuracy_score(y_test,np.round(y_pred))}")
print(f"F1: {f1_score(y_test,np.round(y_pred),average='macro')}")
print(f"ROC-AUC: {roc_auc_score(y_test,y_pred)}")




ACC: 0.7715977644877386
F1: 0.16649414763495118
ROC-AUC: 0.9367596320716931


In [19]:
aucs = []
for i, c in enumerate(config.LIST_CLASSES):
    auc = roc_auc_score(y_test.to_numpy()[:,i],y_pred[:,i])
    print(f"{c}: {auc}")
    aucs.append(auc)
print(np.mean(aucs))

toxic: 0.8905167250122382
severe_toxic: 0.9748855933391991
obscene: 0.924322121638508
threat: 0.9581161025118052
insult: 0.916715842633365
identity_hate: 0.9560014072950426
0.9367596320716931


Same with stopwords removed

In [3]:
X_train_processed = pd.read_csv("../data/X_train.csv",skip_blank_lines=False)
X_test_processed = pd.read_csv("../data/X_test.csv",skip_blank_lines=False)
ts = pp.TokenSequencer()
X_train_seq = ts.fit_transform(X_train_processed)
X_test_seq = ts.transform(X_test_processed)
em = dh.get_embedding_matrix(ts)
model = lstm_multi_label.get_model(embedding_matrix=em)
history = model.fit(X_train_seq, y_train.to_numpy(), 
                    batch_size=config.BATCH_SIZE, 
                    epochs=config.EPOCHS, 
                    validation_split=config.VALIDATION_SPLIT,
                    callbacks=lstm_binary_label.callbacks_list, 
                    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 143613 samples, validate on 15958 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
143584/143613 [============================>.] - ETA: 0s - loss: 0.0603 - acc: 0.9791
Epoch 00001: val_loss improved from inf to 0.04918, saving model to C:\Users\pasca\Dropbox\DataScienceAndProgramming\Projects\tox-block\tox-block\tox_block\trained_models/checkpoint_BL.h5
143613/143613 [==============================] - 920s 6ms/sample - loss: 0.0602 - acc: 0.9791 - val_loss: 0.0492 - val_acc: 0.9815


In [4]:
y_pred = model.predict(X_test_seq)

In [5]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
print(f"ACC: {accuracy_score(y_test,np.round(y_pred))}")
print(f"F1: {f1_score(y_test,np.round(y_pred),average='macro')}")
print(f"ROC-AUC: {roc_auc_score(y_test,y_pred)}")

ACC: 0.7911323809772531
F1: 0.1554343908366093
ROC-AUC: 0.9347452078901068


In [10]:
merged = pd.read_csv(config.MERGED_DATA_FILE)
model = load_model(config.MODEL_PATH)
encoder = joblib.load(config.ENCODER_PATH)

In [40]:
text = pd.DataFrame(["I must frankly say that I have very little to do with this world view...you stupit little fuck nigga"], columns=["comment_text"])
pd.DataFrame(model.predict(encoder.transform(text)), columns=config.LIST_CLASSES)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.532498,0.186156,0.485991,0.014387,0.464562,0.423471


In [56]:
text = merged[merged.identity_hate>0].comment_text.to_frame()
preds = pd.DataFrame(model.predict(encoder.transform(text)), columns=config.LIST_CLASSES)

In [57]:
preds.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,2117.000000,2117.000000,2117.000000,2117.000000,2117.000000,2117.000000
mean,0.470182,0.128319,0.348645,0.040099,0.351372,0.258204
std,0.169333,0.118683,0.214537,0.072784,0.199188,0.153283
min,0.001004,0.000005,0.000181,0.000009,0.000102,0.000005
25%,0.374008,0.016116,0.149161,0.003313,0.181510,0.109851
50%,0.488851,0.101505,0.387762,0.011656,0.374422,0.286828
75%,0.574776,0.216818,0.516338,0.036840,0.509719,0.395673
max,0.933756,0.477057,0.900049,0.537239,0.852248,0.497744


In [45]:
def rescale_proba(p: float, min: float, max: float):
    return (p - min) / (max - min)

In [84]:
text = pd.DataFrame(["fucking Jew"], columns=["comment_text"])
pred = pd.DataFrame(model.predict(encoder.transform(text)), columns=config.LIST_CLASSES)
rescale_proba(pred.identity_hate,0.000005,0.497744)
#pred.identity_hate

0    0.615287
Name: identity_hate, dtype: float32